# Experiment 02: Deformations Experiments ETH-01

In this notebook, we are using the CLUST Dataset.
The sequence used for this notebook is ETH-01.zip


In [1]:
import sys
import random
import os
sys.path.append('../src')
import warnings
warnings.filterwarnings("ignore") 
from PIL import Image
from sklearn.manifold import Isomap
from utils.compute_metrics import get_metrics, get_majority_vote,log_test_metrics
from utils.split import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold
from tqdm import tqdm
from pprint import pprint
import torch
from itertools import product
import pickle
import pandas as pd
import numpy as np
import mlflow
import matplotlib.pyplot as plt
#from kymatio.numpy import Scattering2D

import torch
from tqdm import tqdm
from kymatio.torch import Scattering2D
import time
import kymatio.scattering2d.backend as backend

###############################################################################
# Finally, we import the `Scattering2D` class that computes the scattering
# transform.

from kymatio import Scattering2D


# 1. Visualize Sequence of US
We are visualizing the first images from the sequence ETH-01-1 that contains 3652 US images.

In [2]:
directory=os.listdir('../data/02_interim/Data1')
directory.sort()

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 2. Create Dataset

In [3]:
M = 224
N = 224

In [4]:
%%time
batch_size = 128
ll_imgstemp = [plt.imread("../data/02_interim/Data1/" + dir) for dir in directory[0:batch_size]]

CPU times: user 272 ms, sys: 115 ms, total: 386 ms
Wall time: 733 ms


In [5]:
%%time
#ll_imgs = [np.array(Image.open("../data/02_interim/Data1/" + dir).resize(size=(M, N)), dtype='float32') for dir in directory]
ll_imgs = [np.array(Image.open("../data/02_interim/Data1/" + dir), dtype='float32') for dir in directory]

CPU times: user 2.89 s, sys: 1.59 s, total: 4.48 s
Wall time: 9.39 s


In [6]:
%%time
ll_imgs2 = [img.reshape(1,img.shape[0],img.shape[1]) for img in ll_imgs]

CPU times: user 4.86 ms, sys: 452 µs, total: 5.32 ms
Wall time: 5.24 ms


In [7]:
# dataset = pd.DataFrame([torch.tensor(ll_imgs).view(1,M,N).type(torch.float32)], columns='img')
dataset = pd.DataFrame({'img':ll_imgs2}).reset_index().rename(columns={'index':'order'})
# dataset = pd.DataFrame({'img':ll_imgs}).reset_index().rename(columns={'index':'order'})

# 3. Scattering Speed Test

In [8]:
# M,N = dataset['img'].iloc[0].shape[1], dataset['img'].iloc[0].shape[2]
# print(M,N)

In [9]:
#pip install scikit-cuda cupy

In [10]:
pip install pycuda

  Using cached pycuda-2020.1.tar.gz (1.6 MB)
  ERROR: Command errored out with exit status 1:
   command: /home/mila/g/gauthies/.conda/envs/ultra/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-dx0cmdvb/pycuda/setup.py'"'"'; __file__='"'"'/tmp/pip-install-dx0cmdvb/pycuda/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-j4fb4z4t
       cwd: /tmp/pip-install-dx0cmdvb/pycuda/
  Complete output (106 lines):
  ***************************************************************
  *** WARNING: nvcc not in path.
  *** May need to set CUDA_INC_DIR for installation to succeed.
  ***************************************************************
  *************************************************************
  *** I have detected that you have not run configure.py.
  ********************************************

In [13]:
# From: https://github.com/kymatio/kymatio/blob/0.1.X/examples/2d/compute_speed.py
# Benchmark setup
# --------------------
J = 3
L = 8
times = 10
devices = ['cpu', 'gpu']
scattering = Scattering2D(J, shape=(M, N), L=L, backend='torch_skcuda')
data = np.concatenate(dataset['img'],axis=0)
data = torch.from_numpy(data)
x = data[0:batch_size]

/home/mila/g/gauthies/.conda/envs/ultra/lib/python3.7/site-packages/kymatio/frontend/entry.py:21: DeprecationWarning: Torch frontend is currently the default, but NumPy will become the default in the next version.
  " version.", DeprecationWarning)


AttributeError: type object 'cupy.core.core.Indexer' has no attribute '__reduce_cython__'

In [11]:
pip install scikit-cuda

  Using cached scikit_cuda-0.5.3-py2.py3-none-any.whl (114 kB)
  Using cached pycuda-2020.1.tar.gz (1.6 MB)
  ERROR: Command errored out with exit status 1:
   command: /home/mila/g/gauthies/.conda/envs/ultra/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-f5iir0je/pycuda/setup.py'"'"'; __file__='"'"'/tmp/pip-install-f5iir0je/pycuda/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-slzj7fbt
       cwd: /tmp/pip-install-f5iir0je/pycuda/
  Complete output (106 lines):
  ***************************************************************
  *** WARNING: nvcc not in path.
  *** May need to set CUDA_INC_DIR for installation to succeed.
  ***************************************************************
  *************************************************************
  *** I have detected that you have not r

In [71]:
%%time
#mlflow.set_experiment('compute_speed_scattering')
for device in devices:
    #with mlflow.start_run():
        fmt_str = '==> Testing Float32 with {} backend, on {}, forward'
        print(fmt_str.format('torch', device.upper()))

        if device == 'gpu':
            scattering.cuda()
            x = x.cuda()
        else:
            scattering.cpu()
            x = x.cpu()

        scattering.forward(x)

        if device == 'gpu':
            torch.cuda.synchronize()

        t_start = time.time()
        for _ in range(times):
            scattering.forward(x)

        if device == 'gpu':
            torch.cuda.synchronize()

        t_elapsed = time.time() - t_start

        fmt_str = 'Elapsed time: {:2f} [s / {:d} evals], avg: {:.2f} (s/batch)'
        print(fmt_str.format(t_elapsed, times, t_elapsed/times))
#         mlflow.log_param('M',M)
#         mlflow.log_param('N',N)
#         mlflow.log_param('Backend', device.upper())
#         mlflow.log_param('J', J)
#         mlflow.log_param('L', L)
#         mlflow.log_param('Batch Size', batch_size)
#         mlflow.log_param('Times', times)
#         mlflow.log_metric('Elapsed Time', t_elapsed)
#         mlflow.log_metric('Average Time', times)

    ###############################################################################
    # The resulting output should be something like
    #
    # .. code-block:: text
    #
    #   ==> Testing Float32 with torch backend, on CPU, forward
    #   Elapsed time: 624.910853 [s / 10 evals], avg: 62.49 (s/batch)
    #   ==> Testing Float32 with torch backend, on GPU, forward


==> Testing Float32 with torch backend, on CPU, forward
Elapsed time: 523.081820 [s / 10 evals], avg: 52.31 (s/batch)
==> Testing Float32 with torch backend, on GPU, forward
Elapsed time: 16.777041 [s / 10 evals], avg: 1.68 (s/batch)
CPU times: user 53min 2s, sys: 4min 47s, total: 57min 50s
Wall time: 9min 54s
